# speech recognition

In [1]:
#IMPORTING LIBRARIES THAT ARE NEEDED FOR DATA PROCESSING AND PREDICTION

import tensorflow as tf
import librosa
import collections
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.io import wavfile
import warnings
import pandas as pd
import os
import glob
warnings.filterwarnings("ignore")
import scipy.io.wavfile as wav
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

#### using os.walk to get the file names from the directory --> "speech_commands_v0.01"
    In this file names are different types of speech commands

In [2]:
# GETTING THE LABLE FROM THE FOLDERS
sound_names = [dir1 for root,dir1,files in os.walk("speech_commands_v0.01")]
sound_names[0]

['bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'four',
 'go',
 'happy',
 'house',
 'left',
 'marvin',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'wow',
 'yes',
 'zero',
 '_background_noise_']

#### Using glob.glob function:
    To get files from the given speech command folder

In [3]:
#function to get every wav file from the folder
def get_files(sound_word):
    words = [i for i in glob.glob("speech_commands_v0.01//"+sound_word+"//*.wav*")]
    return words

#### Using Librosa:
    --> Librosa is a library used to handle the audio files.
    --> We can use scipy.io.wavfile to read and handle the wav audio files. 
    --> Librosa is used here because it came with default resample condition. It resamples the audio file with 22050Hz. And it changes every audio channels into Mono. so Librosa reduces the feature engineering time and avoid the unwanted codes. because for prediction we have to use mono channel and same sample rate with all the files.
    --> The loaded audio is converted to a float with amplitude values lying in the range of  [−1,1] .
    --> unlike Librosa, scipy.io.wavfile is giving audio data and sample rate of an audio

#### using MFCC:
    --> MFCC - Mel frequency ceptral coefficient
    --> MFCC giving the important audio data.
    --> ceptrum is known as spectrum of spectrum
            c(x(t)) = F^(-1)[log(F(x(t))]

In [4]:
# FUNCTION TO GET IMPORTANT FEATURE FROM THE AUDIO USING MFCC (mel frequency ceptral coefficient)
def get_data(word_sound,recordings,label,feature):
    for i in word_sound:
        data_sample, sample_rate = librosa.load(i,res_type = 'kaiser_fast')
        mfccs = librosa.feature.mfcc(y = data_sample, sr = sample_rate, n_mfcc = 40)
        mfcc_scaled = np.mean(mfccs.T,axis = 0)
        recordings.append(mfcc_scaled)
        label.append(feature)
    return (recordings,label)

In [5]:
# initialize recordings and labels list to store audio data and corresponding labels
recordings = []
labels = []

In [6]:
# getting audios of first five files
for i in sound_names[0][:5]:
    wav_file_list = get_files(i)
    recordings,labels = get_data(wav_file_list,recordings,labels,i)

In [7]:
# convert them into an array
x = np.array(recordings)
y = np.array(labels)

#### using Counter:
    --> Counter function from collection class is used to give the frquency of the given list of values
    --> here Counter is used to find out whether data is imbalanced or not.
    --> the data we got from the audio files are not imbalanced. 
    --> if it is , then we have to use Down sampling or over sampling using imblearn library.

In [8]:
#checking the lables is biased or not

from collections import Counter
Counter(y)

Counter({'bed': 1712, 'bird': 1731, 'cat': 1733, 'dog': 1746, 'down': 2359})

In [9]:
# since it is the multiclass classification problem label encoding on the lables is necessary
y = pd.get_dummies(y)

In [10]:
y.shape

(9281, 5)

In [11]:
# getting training and test data set using train test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 42)

In [12]:
x_train.shape,x_test.shape

((6496, 40), (2785, 40))

#### using tensorflow:
    --> tensorflow.keras is used to create Neural Nets.
    --> here we created te sequential model with two dense layers and using RELU as an activation function.
    --> input_shape is given as (40,) which is the shape of the x_train.
    --> since it is a multiclass classification we are using softmax as an activation function in the output layer.
    --> category_crossentropy is used here because it is a multiclass classification problem and Adam optimizer is used because 
    it is work well with noice problems and it is faster than SGD.

In [13]:
# importing tensorflow libraries to create NN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn import metrics

In [14]:
# creating layers of neural networks
model = Sequential()

#layer 1
model.add(Dense(100,input_shape = (40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))


#layer 3 
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.5))


#output layer
model.add(Dense(5))
model.add(Activation('softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               15150     
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 7

In [16]:
model.compile(loss = 'categorical_crossentropy',metrics = ['accuracy'],optimizer = 'adam')

In [20]:
# fit the data agaist the validation set
num_epochs = 100
batch_size = 64
k = model.fit(x_train,y_train,batch_size = batch_size, epochs = num_epochs,
         validation_data = (x_test,y_test))

Epoch 1/100
102/102 [==============================] - 1s 7ms/step - loss: 0.8402 - accuracy: 0.6838 - val_loss: 0.8939 - val_accuracy: 0.6664
Epoch 2/100
102/102 [==============================] - 1s 6ms/step - loss: 0.8229 - accuracy: 0.6861 - val_loss: 0.8916 - val_accuracy: 0.6625
Epoch 3/100
102/102 [==============================] - 1s 6ms/step - loss: 0.8306 - accuracy: 0.6904 - val_loss: 0.9002 - val_accuracy: 0.6700
Epoch 4/100
102/102 [==============================] - 1s 6ms/step - loss: 0.8362 - accuracy: 0.6810 - val_loss: 0.8967 - val_accuracy: 0.6600
Epoch 5/100
102/102 [==============================] - 1s 6ms/step - loss: 0.8346 - accuracy: 0.6800 - val_loss: 0.8886 - val_accuracy: 0.6679
Epoch 6/100
102/102 [==============================] - 1s 7ms/step - loss: 0.8331 - accuracy: 0.6824 - val_loss: 0.8906 - val_accuracy: 0.6675
Epoch 7/100
102/102 [==============================] - 1s 6ms/step - loss: 0.8253 - accuracy: 0.6880 - val_loss: 0.8968 - val_accuracy: 0.6625

102/102 [==============================] - 1s 8ms/step - loss: 0.8156 - accuracy: 0.6964 - val_loss: 0.9018 - val_accuracy: 0.6686
Epoch 58/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8299 - accuracy: 0.6923 - val_loss: 0.8960 - val_accuracy: 0.6722
Epoch 59/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8396 - accuracy: 0.6847 - val_loss: 0.9033 - val_accuracy: 0.6621
Epoch 60/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8325 - accuracy: 0.6949 - val_loss: 0.8924 - val_accuracy: 0.6657
Epoch 61/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8253 - accuracy: 0.6926 - val_loss: 0.8847 - val_accuracy: 0.6715
Epoch 62/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8171 - accuracy: 0.6884 - val_loss: 0.8934 - val_accuracy: 0.6632
Epoch 63/100
102/102 [==============================] - 1s 8ms/step - loss: 0.8132 - accuracy: 0.6913 - val_loss: 0.8952 - val_accuracy: 0.6693
Epoch

In [30]:
test_accuracy = model.evaluate(x_test,y_test,verbose = 0)
print(test_accuracy[1])

0.6646319627761841
